In [2]:
import numpy as np
H = 400
W = 400
f = 1200

In [3]:
rays_o =  np.zeros((H*W,3)) #Because we are in 3D
rays_d = np.zeros((H*W,3))

u = np.arange(W)
v = np.arange(H)

In [6]:
u,v

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [7]:
u,v = np.meshgrid(u,v)

In [8]:
u

array([[  0,   1,   2, ..., 397, 398, 399],
       [  0,   1,   2, ..., 397, 398, 399],
       [  0,   1,   2, ..., 397, 398, 399],
       ...,
       [  0,   1,   2, ..., 397, 398, 399],
       [  0,   1,   2, ..., 397, 398, 399],
       [  0,   1,   2, ..., 397, 398, 399]])

In [9]:
v

array([[  0,   0,   0, ...,   0,   0,   0],
       [  1,   1,   1, ...,   1,   1,   1],
       [  2,   2,   2, ...,   2,   2,   2],
       ...,
       [397, 397, 397, ..., 397, 397, 397],
       [398, 398, 398, ..., 398, 398, 398],
       [399, 399, 399, ..., 399, 399, 399]])

In [12]:
dirs = np.stack((u - W/2, - (v - H/2), - np.ones_like(u) * f), axis=-1)

In [13]:
dirs.shape

(400, 400, 3)

In [14]:
rays_d = dirs / np.linalg.norm(dirs, axis=-1, keepdims=True)

In [15]:
rays_d

array([[[-0.16222142,  0.16222142, -0.97332853],
        [-0.1614315 ,  0.16224272, -0.9734563 ],
        [-0.16064128,  0.16226391, -0.97358349],
        ...,
        [ 0.15985074,  0.16228501, -0.97371008],
        [ 0.16064128,  0.16226391, -0.97358349],
        [ 0.1614315 ,  0.16224272, -0.9734563 ]],

       [[-0.16224272,  0.1614315 , -0.9734563 ],
        [-0.1614527 ,  0.1614527 , -0.97358413],
        [-0.16066237,  0.1614738 , -0.97371136],
        ...,
        [ 0.15987174,  0.1614948 , -0.973838  ],
        [ 0.16066237,  0.1614738 , -0.97371136],
        [ 0.1614527 ,  0.1614527 , -0.97358413]],

       [[-0.16226391,  0.16064128, -0.97358349],
        [-0.1614738 ,  0.16066237, -0.97371136],
        [-0.16068338,  0.16068338, -0.97383864],
        ...,
        [ 0.15989264,  0.16070428, -0.97396534],
        [ 0.16068338,  0.16068338, -0.97383864],
        [ 0.1614738 ,  0.16066237, -0.97371136]],

       ...,

       [[-0.16228501, -0.15985074, -0.97371008],
        [-0